In [2]:
###
### Import Libraries
###

from py2neo import Graph, Node, Relationship, authenticate

In [3]:
###
### First Authenticate with your neo4j local server
###

authenticate("localhost", "neo4j", "Jesusislife1!")

In [4]:
###
### Create a graph object on your server
###

g = Graph(host="localhost")
print type(g)

In [27]:
###
### Execute some inline commands to see what our data looks like
###

!ls

!head node_city.txt

edge_distance.txt node_city.txt     testneo4j.ipynb
Ney_York, 12
Chicago, 23
Las_Angelas, 35
Boston, 2

In [28]:
###
### Begin a transaction
### For each row, make it a node, specifying as many properties as you wish in the arguments
### Be sure to specify what kind of Node() you want, for our example, we want a City Node().
### After we are done, we commit the changed to the graph.

tx = g.begin()
with open("node_city.txt") as f_in:
    for line in f_in:
        city_list = line.rstrip().split(",")
        city = Node("City", name=city_list[0], population=int(city_list[1]))
        tx.create(city)
tx.commit()

In [34]:
###
### Execute a MATCH query, for every a in City return its name and population
### This returns a subset of nodes on your graph that you want to see based on the query
###

g.data("MATCH (a : City) RETURN a.name,a.population")

[{u'a.name': u'Ney_York', u'a.population': 12},
 {u'a.name': u'Chicago', u'a.population': 23},
 {u'a.name': u'Las_Angelas', u'a.population': 35},
 {u'a.name': u'Boston', u'a.population': 2},
 {u'a.name': u'Ney_York', u'a.population': 12},
 {u'a.name': u'Chicago', u'a.population': 23},
 {u'a.name': u'Las_Angelas', u'a.population': 35},
 {u'a.name': u'Boston', u'a.population': 2}]

In [60]:
###
### Get a result set, which is a collection of nodes with the name specified
###

result_set = g.node_selector.select("City")
print type(result_set)
print result_set.first()

<class 'py2neo.database.selection.NodeSelection'>
(ney_york:City {name:"Ney_York",population:12})


In [36]:
###
### From our result set, get a City Node by property
###

resQuery = result_set.where(population = 2)
for res in resQuery:
    print res

(boston:City {name:"Boston",population:2})
(boston:City {name:"Boston",population:2})


In [61]:
###
### Print the result set a couple different ways
###

print "All Cities: "
for r in result_set:
    print r
    #print r.items() #Returns a cool list of tuples

print

print "Cities with population greater than 15: "
for r in result_set:
    if r.items()[1][1] >= 15:
        print r.items()[0]

All Cities: 
(ney_york:City {name:"Ney_York",population:12})
(b03164d:City {name:"Chicago",population:23})
(e1ebb18:City {name:"Las_Angelas",population:35})
(boston:City {name:"Boston",population:2})
(c729341:City {name:"Ney_York",population:12})
(fb8f6a4:City {name:"Chicago",population:23})
(de7fbf4:City {name:"Las_Angelas",population:35})
(boston:City {name:"Boston",population:2})

Cities with population greater than 15: 
(u'name', u'Chicago')
(u'name', u'Las_Angelas')
(u'name', u'Chicago')
(u'name', u'Las_Angelas')


In [59]:
###
### Make sure our edge distances are solid
###

!head edge_distance.txt

Boston, Ney_York, 2
Ney_York, Chicago, 10
Chicago, Las_Angelas, 100
Las_Angelas, Boston, 105

In [109]:
###
### Create a relationship link and add it to our graph
### The idea behind this is the following:
###     1. Get a result set containing everything in our Node Graph
###     2. Clean the data into a list of objects
###     3. Find where the Nodes exist in the graph, return their node object references
###     4. Create a Relationship(to, "description", from) object with both node references as args 'to' 'from'
###     5. Add that relationship to the graph i.e., tx.create(yourRelationship)
###
### Pro-tip: You can add new features to the relationship object just by adding it to the dictionary
###

tx = g.begin()

with open("edge_distance.txt") as f_in:
    for line in f_in:
        edge_list = line.rstrip().split(",")

        city1 = edge_list[0].strip()
        city2 = edge_list[1].strip()
        print city1, "\t", city2
        city1_node = result_set.where(name=city1)
        city2_node = result_set.where(name=city2)
        print city1_node.first(), "\t", city2_node.first() 
        city_pair = Relationship(city1_node, "FLIGHT_BETWEEN" ,city2_node)
        city_pair["distance"] = edge_list[2].strip()
        print city_pair.items()
        tx.create(city_pair)

tx.commit()

Boston 	Ney_York
(a1ec03b:City {name:"Boston",population:2}) 	(ney_york:City {name:"Ney_York",population:12})
[('distance', u'2')]
Ney_York 	Chicago
(ney_york:City {name:"Ney_York",population:12}) 	(fcffb19:City {name:"Chicago",population:23})
[('distance', u'10')]
Chicago 	Las_Angelas
(fcffb19:City {name:"Chicago",population:23}) 	(aa9088a:City {name:"Las_Angelas",population:35})
[('distance', u'100')]
Las_Angelas 	Boston
(aa9088a:City {name:"Las_Angelas",population:35}) 	(bfc72aa:City {name:"Boston",population:2})
[('distance', u'105')]


In [110]:
g.relationship_types

frozenset({u'FLIGHT BETWEEN', u'FLIGHT_BETWEEN'})

In [111]:
import pandas as pd

In [116]:
flights = list()
for cities in g.match(rel_type='FLIGHT_BETWEEN'):
    city1_name = cities.nodes()[0]['name']
    city2_name = cities.nodes()[1]['name']
    distance = cities['distance']
    print city1_name, "\t", city2_name, "\t", distance
    flights.append([city1_name, city2_name, distance])

None 	None
None 	None
None 	None
None 	None
None 	None
None 	None
None 	None
None 	None
None 	None
None 	None
None 	None
None 	None
Boston 	None
Ney_York 	None
Chicago 	None
Las_Angelas 	None
Boston 	None
Ney_York 	None
Chicago 	None
Las_Angelas 	None
Boston 	None
Ney_York 	None
Chicago 	None
Las_Angelas 	None
Boston 	None
Ney_York 	None
Chicago 	None
Las_Angelas 	None
Boston 	Ney_York
Ney_York 	Chicago
Chicago 	Las_Angelas
Las_Angelas 	Boston


In [119]:
flights_df = pd.DataFrame(flights)
flights_df.columns = ["to", "from", "distance"]
flights_df.tail()

,to,from,distance
27,Las_Angelas,None,105
28,Boston,Ney_York,2
29,Ney_York,Chicago,10
30,Chicago,Las_Angelas,100
31,Las_Angelas,Boston,105
